In [35]:
!pip install langchain==0.3.30
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14
!pip install langchain-huggingface==0.1.2

ERROR: Could not find a version that satisfies the requirement langchain==0.3.30 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.0.26, 0.0.27, 0.0.28, 0.0.29, 0.0.30, 0.0.31, 0.0.32, 0.0.33, 0.0.34, 0.0.35, 0.0.36, 0.0.37, 0.0.38, 0.0.39, 0.0.40, 0.0.41, 0.0.42, 0.0.43, 0.0.44, 0.0.45, 0.0.46, 0.0.47, 0.0.48, 0.0.49, 0.0.50, 0.0.51, 0.0.52, 0.0.53, 0.0.54, 0.0.55, 0.0.56, 0.0.57, 0.0.58, 0.0.59, 0.0.60, 0.0.61, 0.0.63, 0.0.64, 0.0.65, 0.0.66, 0.0.67, 0.0.68, 0.0.69, 0.0.70, 0.0.71, 0.0.72, 0.0.73, 0.0.74, 0.0.75, 0.0.76, 0.0.77, 0.0.78, 0.0.79, 0.0.80, 0.0.81, 0.0.82, 0.0.83, 0.0.84, 0.0.85, 0.0.86, 0.0.87, 0.0.88, 0.0.89, 0.0.90, 0.0.91, 0.0.92, 0.0.93, 0.0.94, 0.0.95, 0.0.96, 0.0.97, 0.0.98, 0.0.99rc0, 0.0.99, 0.0.100, 0.0.101rc0, 0.0.101, 0.0.102rc0, 0.0.102, 0.0.103, 0.0.104, 0.0.105, 0.0.106, 0.0.107, 0.0.108, 0.0.109, 0.0

In [36]:
!pip install langchain-google-genai

In [37]:
from getpass import getpass

In [38]:
api_key = getpass("enter your gemini api key")

enter your gemini api key··········


**Upload and Preprocess video using Gemini API**

Next, use the File API to upload the video file.

In [39]:
import google.generativeai as genai

In [40]:
video_path = "/content/12577525-uhd_3840_2160_30fps.mp4"

In [41]:
genai.configure(api_key= api_key)

In [42]:
#uploading the file
file = genai.upload_file(path = video_path)

In [43]:
print(f"Upload complete: {file.uri}")

Upload complete: https://generativelanguage.googleapis.com/v1beta/files/54tku45knyyi


After uploading the file, you can call `get_file` to verify if the API has successfully processed the file.

`get_file` allows you to check the uploaded file associated with the File API in the cloud project linked to the API key.

In [44]:
import time
while file.state.name == "PROCESSING":
  print("please wait the the video is processing...")
  time.sleep(5)
  file = genai.get_file(file.name)
if file.state.name == "FAILED":
  raise ValueError(file.name)

# Print completion message
print(
    f"\nVideo processing is complete!\nYou can now start the conversation: {file.uri}"
)

please wait the the video is processing...
please wait the the video is processing...

Video processing is complete!
You can now start the conversation: https://generativelanguage.googleapis.com/v1beta/files/54tku45knyyi


**Generate content (Gemini API)**

After the video is preprocessed, you can use the `generate_content` function from Gemini API to request questions about the video.

In [45]:
prompt = "describe the given video clips"

In [46]:
model = genai.GenerativeModel(model_name= "models/gemini-1.5-flash")

In [47]:
response = model.generate_content(
    [prompt, file], request_options={"timeout": 600}
)

In [48]:
print(response.text)

The video shows an aerial view of a section of city located on the edge of a body of water, likely a lake. 


Here's a breakdown of what's visible:

* **Waterfront:** The video opens with a view of a calm, teal-colored body of water. A paved walkway or promenade runs along the shoreline.

* **Park-like Area:** A well-maintained, landscaped park with geometric designs is situated near the water's edge.  It features pathways and lush greenery.

* **Buildings:**  The majority of the video focuses on various buildings of different sizes and styles, suggesting a mix of residential and potentially commercial structures. Many buildings have brown tiled roofs, characteristic of European architecture. A large, light-colored building with classical features is prominent near the intersection.

* **Roads and Transportation:** Roads curve along the waterfront and extend into the city.  Buses and what appears to be a tram or light rail system are visible, suggesting public transportation is in plac

**Integrating the Gemini Model into a LangChain Workflow for Video Data**

Here is an example of using LangChain with the Gemini model.

The model is loaded via `ChatGoogleGenerativeAI` from `langchain_google_genai`, allowing multimodal data to be included in the content of `HumanMessage `using the `media `type.

In [ ]:
!pip install langchain-google-genai

In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [50]:
from langchain_core.messages import HumanMessage

In [51]:
llm = ChatGoogleGenerativeAI(model = "models/gemini-1.5-flash", api_key= api_key)

In [54]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key= api_key)

# Create a message to send to the model and attach the video file as media input
message = HumanMessage(
    content=[
        {"type": "text", "text": "Please analyze the content of this video."},
        {
            "type": "media",
            "mime_type": file.mime_type,
            "file_uri": file.uri,
        },
    ]
)

In [55]:

# Stream the response and process each chunk
for chunk in llm.stream([message]):
    print(chunk.content)

This
 video shows an aerial view of a section of the city of Montreux, Switzerland
, situated on the shores of Lake Geneva. 


Here's a breakdown
 of the content:

* **Location:** The primary focus is on a waterfront area with a curved road following the shoreline. The camera slowly pans out,
 revealing more of the city's architecture and layout.  The presence of Lake Geneva is prominent throughout the video.

* **Architecture:** The video displays a
 mix of architectural styles, mostly European.  Older, smaller buildings with traditional roofs are visible alongside more modern structures. One large, light-colored building near the center is particularly noticeable.

* **Infrastructure:** Roads, a tramway or light
 rail line, and bus stops are clearly seen. The infrastructure is well-organized and integrated into the city's design.

* **Landscape:** The greenery of trees and well-maintained landscaping is integrated throughout the city, particularly in
 a park-like area near the lake.
